In [46]:
# plots
%matplotlib inline

import matplotlib.pyplot as plt
from pylab import rcParams
import math
rcParams['figure.figsize'] = 15, 8

import math
import scipy

import json

# Pandas
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from pandas import DataFrame

# math api
import numpy as np

# time lib
import time
import datetime as dt

NOW = dt.datetime.now()


import seaborn as sns
sns.set()




In [47]:
flat_data = pd.read_csv("../ressources/data/financial_sells_100000.csv")
flat_data = flat_data.fillna(0).replace('-', np.NaN)
flat_data = flat_data.drop(columns=['Unnamed: 0'])
flat_data.head()

,company_name,company_id,country,sector_name,order_id,product_name,NBI,order_date
0,ICBC,1,Espagne,Electrical industry,1,International Retail Banking,-0.499521,2019-05-29
1,HSBC Holdings,2,United Kinkdom,Chemical Industry,3,Financing,109.972460,2018-09-09
2,Alibaba,3,United State,Energy industry,4,General Purpose Financing,2.868872,2018-02-23
3,Novartis,4,Italy,Metallurgical industry,5,Commodities,38.456910,2016-11-04
4,Johnson & Johnson,6,Suisse,Electrical industry,7,Life and Dammage insurance,51.705999,2018-08-25


# first implementation of Cross-Selling

In [33]:
dict_combinaison = dict()

for sector in flat_data['sector_name'].unique(): 
    for country in flat_data['country'].unique():
        
        
        query_fetch = 'country == "{}" and sector_name == "{}"'.format(country, sector)
        result_query = flat_data.query(query_fetch)
        all_products = result_query['product_name'].unique()
        
        for company_name in result_query['company_name'].unique():
            bought_products = set(flat_data.query('company_name == "{}"'.format(company_name))['product_name'].unique())
            all_products = set(all_products)
            unbought_products = all_products.symmetric_difference(bought_products)
            
            if( len(unbought_products) > 0 ):
                dict_combinaison[ len(dict_combinaison) ] = {
                    'company_name': company_name,
                    'sector': sector,
                    'country': str(country),
                    'bought_product': str(bought_products),
                    'unbought_product': str(unbought_products),
                    'diff': len(unbought_products)
                }



print(dict_combinaison.get(0))
print(len(dict_combinaison))
flat_data.query(' company_name == "Powers-Woodard" and country == "Espagne" ')



{'company_name': 'Nestle', 'sector': 'Electrical industry', 'country': 'Italy', 'bought_product': "{'Life and Dammage insurance', 'ALD Car Renting and Fleet Management', 'Others', 'International Retail Banking'}", 'unbought_product': "{'Leverage Finance'}", 'diff': 1}
11


,company_name,company_id,country,sector_name,order_id,product_name,NBI,order_date


In [136]:
result_df = pd.DataFrame()
for value in dict_combinaison.values():
    result_df = result_df.append(value, ignore_index=True)
    
result_df.to_csv('../ressources/data/2_cross_selling_result.csv', sep=';')


# second implementation of cross_selling

In [152]:
def apply_cross_selling(_df):
    # count all unique clients
    count_distinct_customers = _df['company_name'].nunique()

    result_cross_selling = dict()

    # for each item, compare with otel items
    for product_1 in _df['product_name'].unique(): 
        
        #print( product_1 )
        
        #count all users who bougth this item1
        query_product_1 = "product_name == '{0}'".format(product_1)
        result_query_1 = _df.query(query_product_1)


        for product_2 in _df['product_name'].unique():
            dict_key1 = (product_2, product_1)
            dict_key2 = (product_1, product_2)
            score = 0

            if ( dict_key1 in result_cross_selling ):
                #print('already exists', end=' - ')
                continue

            #count all users who bougth this item1
            query_product_2 = "product_name == '{0}'".format(product_2)
            result_query_2 = _df.query(query_product_2)

            list_customer_product_1 = set( result_query_1['company_name'].unique() )
            list_customer_product_2 = set( result_query_2['company_name'].unique() )
            common_customers = list_customer_product_1.intersection(list_customer_product_2)

            score = len( common_customers ) / count_distinct_customers

            

            result_cross_selling[ dict_key1 ] = score
            result_cross_selling[ dict_key2 ] = score
            
        return result_cross_selling

    

result_cross_sells = apply_cross_selling(flat_data)

In [153]:
array_cross_sells = [[k[0], k[1], v] for k, v in result_cross_sells.items()]
cross_pd = pd.DataFrame(array_cross_sells, columns=['product_1', 'product_2', 'coef'])

cross_pd

,product_1,product_2,coef
0,International Retail Banking,International Retail Banking,0.56
1,Financing,International Retail Banking,0.16
2,International Retail Banking,Financing,0.16
3,General Purpose Financing,International Retail Banking,0.36
4,International Retail Banking,General Purpose Financing,0.36
5,Commodities,International Retail Banking,0.18
6,International Retail Banking,Commodities,0.18
7,Life and Dammage insurance,International Retail Banking,0.14
8,International Retail Banking,Life and Dammage insurance,0.14
9,Issuer Services,International Retail Banking,0.16


# third implementation

[]